In [84]:
# import some nessary libraries
import nltk
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from __future__ import unicode_literals
from hazm import Normalizer, Lemmatizer
from hazm import sent_tokenize, word_tokenize
from hazm import Lemmatizer
from sklearn import model_selection

In [85]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from nltk.classify.scikitlearn import SklearnClassifier


In [86]:
# read data
df = pd.read_table('emails.csv', header=None, encoding='utf-8')

In [87]:
# cleaning data
lables = []
for email in df[0]:
  lables.append(email.split(',')[-1])
lables.remove(lables[0])
print(lables[1])

ham


In [88]:
text = df[0].copy()
for i in range(len(text)):
  if i == 0:
    continue
  text[i] = text[i].split(',')[:-1][0]
text.pop(0)

'text,label'

In [89]:
# encoding lables
encoder = LabelEncoder()
Y = encoder.fit_transform(lables)

In [90]:
# cleaning text
processed = text.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$',
                                 ' emailaddress ',regex=True)
processed = processed.str.replace(r'(http|ftp|https):\/\/[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?',
                                  ' webaddress ',regex=True)
processed = processed.str.replace(r'تومان|ریال|دلار|\$|تومن|هزار|میلیون|یورو', ' moneysymb ',regex=True)

processed = processed.str.replace(r'^\(?[\d]{3}\)?[\s-]?[\d]{3}[\s-]?[\d]{4}$',
                                  ' phonenumbr ',regex=True)
processed = processed.str.replace(r'\d+(\.\d+)?', ' numbr ',regex=True)

processed = processed.str.replace(r'[^\w\d\s]', ' ',regex=True)

processed = processed.str.replace(r'\s+', ' ',regex=True)

processed = processed.str.replace(r'^\s+|\s+?$', '',regex=True)

processed = processed.str.lower()


In [91]:
# normalization
normalizer = Normalizer()
for i in range(len(processed)):
  if i == 0:
    continue
  processed[i] = normalizer.normalize(processed[i])
  processed[i] = word_tokenize(processed[i])
from hazm.utils import stopwords_list


In [92]:
# stop words
stopwords = set(stopwords_list())
processed = processed.apply(lambda x:[
    term for term in x if term not in stopwords])

In [93]:
# lemmatization
ps = Lemmatizer()
processed = processed.apply(lambda x: [ps.lemmatize(term) for term in x])

In [94]:
all_words = []
for i in processed:
  for j in i:
    all_words.append(j)

In [95]:
all_words = nltk.FreqDist(all_words)
print('Number of words: {}'.format(len(all_words)))
print('Most common words: {}'.format(all_words.most_common(15)))

Number of words: 15108
Most common words: [('numbr', 6142), ('webaddress', 2864), ('moneysymb', 1319), ('داشت#دار', 891), ('قیمت', 837), ('خرید', 817), ('آموزش', 757), ('کرد#کن', 678), ('پست', 662), ('سایت', 584), ('استفاده', 564), ('شد#شو', 525), ('کار', 494), ('دانشگاه', 493), ('صورت', 484)]


In [96]:
word_features = list(all_words.keys())[:1500]


In [97]:
def find_features(message):
    words = message
    features = []
    for word in word_features:
        features.append(int(word in words))
    return features


In [98]:
featuresets = [find_features(text) for text in processed.iloc[1:]]

In [99]:

X,Xt, Y,Yt = model_selection.train_test_split(featuresets,Y[1:], test_size = 0.25, random_state=1)

In [100]:
X, Xt, Y, Yt = np.array(X), np.array(Xt), np.array(Y), np.array(Yt)

In [101]:
classifiers = [
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    LogisticRegression(),
    SGDClassifier(),
    MultinomialNB(),
    SVC()
]
for clf in classifiers:
    clf.fit(X, Y)
    y_pred = clf.predict(Xt)
    print(clf.__class__.__name__ + ':', accuracy_score(Yt, y_pred))


KNeighborsClassifier: 0.736
DecisionTreeClassifier: 0.8
RandomForestClassifier: 0.944
LogisticRegression: 0.96
SGDClassifier: 0.948
MultinomialNB: 0.94
SVC: 0.936
